<a href="https://colab.research.google.com/github/mafux777/kintusgi-extras/blob/kintsugi-x/Kintsugi_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's start with some utilities for substrate for python..

In [77]:
# next line only needed in Google Colaboratory! Otherwise pip install yourself
!pip install substrate-interface

from substrateinterface import SubstrateInterface

kint_substrate = SubstrateInterface(
    url="wss://api-kusama.interlay.io/parachain"
)
ksm_substrate = SubstrateInterface(
    url="wss://kusama-rpc.dwellir.com"
)

def convert_kint_to_ksm(kint):
    universal = kint_substrate.ss58_decode(kint)
    kusama = ksm_substrate.ss58_encode(universal)
    return kusama

def convert_ksm_to_kint(k):
    universal = ksm_substrate.ss58_decode(k)
    kint = kint_substrate.ss58_encode(universal)
    return kint



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 14.3 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 856 kB 62.3 MB/s 
     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 243 kB 77.4 MB/s 
     |████████████████████████████████| 1.0 MB 46.9 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
     |████████████████████████████████| 2.0 MB 51.7 MB/s 
     |████████████████████████████████| 119 kB 72.1 MB/s 
     |████████████████████████████████| 1.0 MB 39.6 MB/s 
     |████████████████████████████████| 481 kB 67.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236746 sha256=3eed24fc02ca8b3648087053dfccd41616f3a2128391eca87030e2bdbf0af6fe
  Stored in directory: /root/.cache/pip/wheel

'a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw'

Let's analyze Kintsugi's vaults and early investors. For that, we will use several subsquids.

Kintsugi-X is a subsquid built by the Kintsugi-X team for the purpose of analysing transfers.

In [41]:
import requests
import json
import pandas as pd

kintsugi = "https://api-kusama.interlay.io/graphql/graphql"
kusama = "https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql"
kintsugi_x = "https://app.gc.subsquid.io/beta/kintsugi-x/v2/graphql"


One of the interesting questions is: do vaults self-mint?

In [42]:
def is_equal(row):
    return row["userParachainAddress"] == row['vault.accountId']


Let's start creating a directory of interesting Kintsugi addresses.
This info comes from the Interlay discord server, "meet the vaults".

In [43]:
kts = {
    "a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs": ['Simon Kraus', '@sik'],
    "a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63": ['Simon Kraus', '@sik'],
    "a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo": ['Simon Kraus', '@sik'],
    "a3bccyaV6tCoqaWqByeqvXo5kBbd1m4yCPKyX937HB1APTt7Y": ['rodrigo.barrios', 'hypersphere'],
    "a3bzFrZ5kXYpaaD5NbapUDSfjZPQTWFKGwSbMmGeRAL8BGrCs": ['@boyswan'],
    "a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6": ['@seergeist'],
    "a3dh62XsvNmtPAzfwCH9bv34dqPFzjKcWBi5mYM93mKmMt64s": ['@DkA7s'],
    "a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK": ['@whisperit'],
    "a3aDPraojQvYhVHjyVuYRFXno58EPMjegrY9nubPmpck2X7JS": ['@blinkin', 'chaos DAO'],
    "a3azGTG3qGmUuQckCKjFAhfjfnnRAXmpgV4fPVPziNaA1zCwG": ['@marvel'],
    "a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS": ['@mafux777'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3baaLbC1JMHJLJ2HwEQMz3S5VuiCWBYy4i66Ziq1vXzmVU6b": ['@spazcoin', 'chaos DAO'],
    "a3fudELrRCjuSyYEPkRAKFQyjzo5YyU228LdqinGsnjBUNB8P": ['@spazcoin', 'chaos DAO', 'VaaS'],
    "a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW": ['@spazcoin', 'chaos DAO'],
    "a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed": ['@spazcoin', 'chaos DAO'],
    "a3aPvmjypKaDtjRgYbDL2CCkseYR3SLwvPevL6j7wF67aFtV4": ['@timbotronic'],
    "a3fZSzXxTZYY58BQrfhJx8cDtp4wRdbZ8X4ReF2iUT63y5RcX": ['@0xvault'],
    "a3dJfVzssBJgBmRuMZBre5H71rvawHJoFvFHGz2Aq7Hdt492w": ['@niko'],
    "a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti": ['@paride'],
    "a3cAyFZMgahPoAyWbNRrjX2TnXQtpS3bztCVMNuLNcTYATBte": ['@dan', 'interlay'],
    "a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe": ['pumpernickel'],
    "a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE": ['@alibaba'],
    "a3cDUVdQi8FqkiUBMjqS3RBWPghRWFXBwjRzKZvmB3MmHLDcP": ['@alibaba'],
    "a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1": ['@warinelly'], # info provided by @quin
}

In [44]:
def add_label_to_list(my_list, label):
  for a in my_list:
    if a not in kts:
      kts[a] = []
    if label not in kts[a]:
      kts[a].append(label)    
  
def enrich_df(df, col):
  df[f'label_{col}'] = df[col].apply(lambda v: "/".join(kts.get(v, [])))

Let's create a query to understand transfers from Kusama (Relay Chain) to Kintsugi. These have to be KSM transfers, because Kusama itself does not handle other assets.
We assume that all of these will end up as collateral. 


In [45]:
to_kintsugi = """
query MyQuery {
  transfers(where: {name_eq: "xcmPallet.reserveTransferAssets", AND: {to: {id_eq: "F7fq1inhrJsYSUkWhyZ3zqtp5K3AKBBjbPWy6VLiRGHipPi"}}}, orderBy: date_DESC) {
    name
    amount
    from {
      id
    }
    date
    to {
      id
    }
  }
}

"""

In [80]:
r = requests.post("https://app.gc.subsquid.io/beta/kusama-explorer/v1/graphql", json={"query" : to_kintsugi}).json()
df_0 = pd.json_normalize(r['data']['transfers'])
df_0['ksm'] = df_0['amount'].apply(lambda x: float(x) / 1e12)
kusama_transfers = df_0.groupby('from.id').agg(dict(ksm="sum", date="min")).sort_values('ksm', ascending=False)
kusama_transfers.reset_index(inplace=True)
kusama_transfers.rename(columns={"from.id": "kusama",
                                 "date": "first_seen"}, inplace=True)
kusama_transfers['kintsugi'] = kusama_transfers.kusama.apply(convert_ksm_to_kint)
kusama_transfers


,kusama,ksm,first_seen,kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2940.17498,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,847.66346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,826.38600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW
...,...,...,...,...
56,J1dYYtLHd4XwdeBrZVELtXxBoabUnXJbfc8oiNvNACpdXqT,0.10000,2022-05-22T02:00:54.028Z,a3feTzhfmfSe3d1RhL1PQe8KGy247afquSZVpy4Bb6pMsphfZ
57,HcNCnh9ZLYRcXB7wm3aPyhzS4i3ghoKP6UR4intuJvp8buf,0.05000,2022-04-25T18:25:06.087Z,a3fFCewUawA7wHtxdRCwkhDVKDHBZnb7vDzN7E4bZdy5sKwfN
58,D8yhHcbwNuYEJUoSDwyQXVnGJzfGpEpnRq8Qdmkc1Etw8vE,0.05000,2022-04-25T19:23:30.004Z,a3amp9SQ3KCV3ugGJufr9hmH73XUBNhZRdKipZyaRLfPx8dqr
59,DoSsDYsK1a8vMaUC3ZhVzvnpKfPZqbBXXQ5cDssxSPgtNqJ,0.01000,2022-05-24T06:51:00.033Z,a3bSHKNLJgq9ebjMyfVTsoEi7bY8ufiunNRHmmZgYh6Yk5gnz


Let's identify all the accounts who crowdloaned to Kintsugi!

In [74]:
crowdloans = """
query CrowloanersKintsugi {
  crowdloans(where: {para: {id_eq: "2092"}}) {
    contributions {
      amount
      timestamp
      account {
        id
        substrateAccount
      }
    }
  }
}
"""
r = requests.post("https://app.gc.subsquid.io/beta/parity-kusama/1/graphql", json={"query" : crowdloans}).json()
crd = pd.json_normalize(r['data']['crowdloans'][0]['contributions'])
crd['ksm'] = crd['amount'].apply(lambda x: float(x) / 1e12)
crowdloans = crd.groupby(['account.id', 'account.substrateAccount']).agg(
    dict(
        ksm=sum,
        timestamp=min
    )
)
crowdloans.describe()

,ksm
count,8013.000000
mean,24.959432
std,486.510228
min,0.100000
25%,0.111000
50%,1.000000
75%,2.100000
max,22688.000000


In [86]:
q = crowdloans.ksm.quantile(.8)
top_loaners = crowdloans[crowdloans.ksm>q]
top_loaners.reset_index(inplace=True)
top_loaners.rename(columns={"account.id":"kusama",
                            "timestamp": "first_seen"}, inplace=True)
top_loaners['kintsugi'] = top_loaners.kusama.apply(convert_ksm_to_kint)

top_loaners.loc[:, ['kusama', 'kintsugi', 'ksm', 'first_seen']].sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,kusama,kintsugi,ksm,first_seen
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,22688.00000,2021-09-24T10:52:42.022Z
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,18517.00000,2021-09-24T10:53:18.003Z
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,17270.44313,2021-09-21T23:42:36.005Z
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,15001.00000,2021-09-17T06:32:06.005Z
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,10000.00000,2021-09-16T11:15:06.007Z
...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,3.05000,2021-09-25T18:32:24.006Z
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,3.05000,2021-09-25T10:54:48.008Z
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,3.04570,2021-09-24T19:57:06.012Z
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,3.04000,2021-09-24T22:29:36.014Z


Now that we know the biggest KSM senders, we should calculate their Kintsugi address. (TO DO IN PYTHON)
(We use a workaround later)

In [48]:
redeem_query = """query MyQuery {
  redeems(orderBy: request_timestamp_ASC) {
    id
    request {
      requestedAmountBacking
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    userBackingAddress
    bridgeFee
    btcTransferFee
    collateralPremium
    status
    execution {
      height {
        absolute
        active
      }
      timestamp
    }
    cancellation {
      timestamp
      slashedCollateral
      reimbursed
      height {
        absolute
        active
      }
    }
  }
}
"""


Let's use the official Kintsugi squid to download data about issue requests.

In [49]:
issue_query = """
query MyQuery {
  issues(orderBy: request_timestamp_DESC, limit: 10000, offset: 0) {
    id
    request {
      amountWrapped
      bridgeFeeWrapped
      timestamp
      height {
        absolute
        active
      }
    }
    userParachainAddress
    vault {
      accountId
      collateralToken
      wrappedToken
    }
    vaultBackingAddress
    vaultWalletPubkey
    griefingCollateral
    status
    refund {
      amountPaid
      btcAddress
      btcFee
      executionHeight {
        absolute
        active
      }
      executionTimestamp
      id
      requestHeight {
        absolute
        active
      }
      requestTimestamp
    }
    execution {
      height {
        absolute
        active
      }
      amountWrapped
      bridgeFeeWrapped
      timestamp
    }
    cancellation {
      timestamp
      height {
        absolute
        active
      }
    }
  },
}
"""

In [50]:
# Obtain a list of all redemptions and summarize them a bit
r = requests.post(kintsugi, json={"query" : redeem_query}).json()
df_1 = pd.json_normalize(r['data']['redeems'])
df_1['btc'] = df_1['request.requestedAmountBacking'].apply(lambda x: float(x) / -1e8)
df_1['self'] = df_1.apply(is_equal, axis=1)
df_1['action'] = "redeem"

# Obtain a list of all issue executions and summarize them a bit
# issue_query = get_query_text_from_file("issue")
r = requests.post(kintsugi, json={"query" : issue_query}).json()
df_2 = pd.json_normalize(r['data']['issues'])
df_2['btc'] = df_2['request.amountWrapped'].apply(lambda x: float(x) / 1e8)
df_2['self'] = df_2.apply(is_equal, axis=1)
df_2['action'] = "issue"

cols_1 = set(df_1.columns) - set(df_2.columns)
cols_2 = set(df_2.columns) - set(df_1.columns)


Let's create a method to add labels to our list.

Let's download data about redemptions, so we can net out issues and redemptions for calculating the vault sizes.
TODO: This analysis should also include Theft and Replacement.
The 80% quantile gives us the accounts responsible for 80% of the issues & redemptions

In [51]:
redeems = df_1.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=True)
redeems['btc'] = redeems.btc.apply(lambda x: -x)
q = redeems.btc.quantile(.8)
top_redeemers = redeems[redeems.btc>q].index
add_label_to_list(top_redeemers, 'Top Redeemer')
redeems['btc'] = redeems.btc.apply(lambda x: -x)


In [52]:
issues = df_2.groupby('userParachainAddress').agg({'btc':sum}).sort_values('btc', ascending=False)
q = issues.btc.quantile(.8)
top_issues = issues[issues.btc>q].index
add_label_to_list(top_issues, 'Top Issuer')
top_issues


Index(['a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE',
       'a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK',
       'a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3',
       'a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq',
       'a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR',
       'a3fCb6j7M4FPqejD1CNTCLDfeoTeg8bR1ooEyqnXkk4srghRp',
       'a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN',
       'a3ai6qqcqHEmRnSwix6Z9tEmsg2uXjFHVtL57oqFEWrLHxK6g',
       'a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK',
       'a3d4WYkJHD1xKTBGaZpr6xJKDngTwtZJ8CqcjDEkKBYtvF469',
       'a3bzJWesF683HSZkvpyXWcejKWVjroyPyF6xGL1GEJBQy5Dh5',
       'a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g',
       'a3awoQLGoQNGvR7UuXvgA7WhEk954SYhLuxvXASenQ8z1LUQH',
       'a3dV3XyhVP3f1B1dwCvW7V8k87M2maqRxLpk8bb5f4TbPVSFD',
       'a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW',
       'a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS',
       'a3dhAVcVNAZAGLvFKZLR2XKRB9xSoh5a

In [53]:
issues[issues.btc>q].btc.sum()

47.11487291

In [54]:
self_issuers = df_2[df_2.self==True]
self_issuers.groupby("userParachainAddress").agg(dict(btc=sum)).sort_values("btc", ascending=False)

,btc
userParachainAddress,
a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,6.453893
a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,3.518315
a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.433175
a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650
a3dh7jXhw2q7vqUpEPAb8BeFTNqXXvG9zBf7hsfm3o9hNbjtK,0.757999
a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,0.734373
a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.601956
a3dCFNgcQsgL4wX6apeaxVLTyRzeCiUdEczyvPcXZ5Dmg5rez,0.510129
a3fcMNTjXcJSwAVnTNKwwP7T8XM2bCW7FshsTW2hpUTrdXzed,0.505341


Let's consolidate this info and merge the two dataframes so we can net out the issue requests and the redeem requests.

In [55]:
merged_df = pd.concat([
    df_1.loc[df_1.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
    df_2.loc[df_2.status=='Completed', ['vault.accountId', 'btc', 'request.timestamp', 'self']],
                       ]).sort_values('request.timestamp')
# apportion BTC to self or other
merged_df['btc_self'] = merged_df.apply(lambda b: b.btc if b.self else 0, axis=1)
merged_df['btc_other'] = merged_df.apply(lambda b: b.btc if not b.self else 0, axis=1)
# add up the BTC depending on self/other
biggest_vaults = merged_df.groupby('vault.accountId').agg(dict(btc_self='sum', btc_other='sum', btc='sum')).sort_values('btc', ascending=False)
biggest_vaults['social'] = biggest_vaults.apply(lambda b: True if b.btc_self<0.2*b.btc else False, axis=1)
biggest_vaults['selfish'] = biggest_vaults.apply(lambda b: True if b.btc_self>0.8*b.btc else False, axis=1)
add_label_to_list(biggest_vaults.loc[biggest_vaults.social].index, 'Social')
add_label_to_list(biggest_vaults.loc[biggest_vaults.selfish].index, 'Selfish')


biggest_vaults.reset_index(inplace=True)
enrich_df(biggest_vaults, "vault.accountId")
total_btc = biggest_vaults.btc.sum()
biggest_vaults['share'] = biggest_vaults.btc.apply(lambda btc: f"{btc/total_btc:.1%}")
biggest_vaults


,vault.accountId,btc_self,btc_other,btc,social,selfish,label_vault.accountId,share
0,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650,12.210787,13.109437,True,False,Top Issuer/Social,45.6%
1,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,4.365793,0.114068,4.479861,False,True,Top Redeemer/Top Issuer/Selfish,15.6%
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,0.000000,1.309054,1.309054,True,False,@warinelly/Social,4.6%
3,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,0.744558,0.183030,0.927588,False,True,Top Redeemer/Top Issuer/Selfish,3.2%
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,-1.738186,2.610267,0.872081,True,False,Top Redeemer/Social,3.0%
5,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.000000,0.625398,0.625398,True,False,@paride/Social,2.2%
6,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.152646,-0.546023,0.606622,False,True,Top Redeemer/Top Issuer/Selfish,2.1%
7,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.601946,-0.018518,0.583428,False,True,@mafux777/Top Issuer/Selfish,2.0%
8,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,0.000000,0.559917,0.559917,True,False,Social,1.9%
9,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,-0.069991,0.583043,0.513053,True,False,pumpernickel/Social,1.8%


In [56]:
vaults = biggest_vaults.reset_index()
del vaults['index']
vaults = vaults.rename(columns={"vault.accountId":"vault"})
add_label_to_list(list(vaults.vault.iloc[0:20]), "Top 20 Vault")
add_label_to_list(list(vaults.vault.loc[vaults.vault.isin(self_issuers.userParachainAddress)].iloc[0:20]), "Self Issuer")
enrich_df(vaults, 'vault')
vaults.iloc[0:25]

,vault,btc_self,btc_other,btc,social,selfish,label_vault.accountId,share,label_vault
0,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,0.898650,12.210787,13.109437,True,False,Top Issuer/Social,45.6%,Top Issuer/Social/Top 20 Vault/Self Issuer
1,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,4.365793,0.114068,4.479861,False,True,Top Redeemer/Top Issuer/Selfish,15.6%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
2,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,0.000000,1.309054,1.309054,True,False,@warinelly/Social,4.6%,@warinelly/Social/Top 20 Vault
3,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,0.744558,0.183030,0.927588,False,True,Top Redeemer/Top Issuer/Selfish,3.2%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
4,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,-1.738186,2.610267,0.872081,True,False,Top Redeemer/Social,3.0%,Top Redeemer/Social/Top 20 Vault/Self Issuer
5,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,0.000000,0.625398,0.625398,True,False,@paride/Social,2.2%,@paride/Social/Top 20 Vault
6,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,1.152646,-0.546023,0.606622,False,True,Top Redeemer/Top Issuer/Selfish,2.1%,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
7,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,0.601946,-0.018518,0.583428,False,True,@mafux777/Top Issuer/Selfish,2.0%,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...
8,a3bihCTc88uby4Xm5D8JbTWszYGhvt34BDKrAamPVikRH8Qem,0.000000,0.559917,0.559917,True,False,Social,1.9%,Social/Top 20 Vault
9,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,-0.069991,0.583043,0.513053,True,False,pumpernickel/Social,1.8%,pumpernickel/Social/Top 20 Vault/Self Issuer


In [57]:
kintsugix_query="""
query MyQuery {
  transfers(orderBy: timestamp_DESC) {
    amount
    from {
      karura
      kintsugi
      kusama
      moonriver
      id
    }
    fromChain
    timestamp
    to {
      id
      karura
      kintsugi
      kusama
      moonriver
    }
    toChain
    token
  }
}

"""

In [58]:
r = requests.post(kintsugi_x, json={"query" : kintsugix_query}).json()
xtoken_transfers = pd.json_normalize(r['data']['transfers'])


In [59]:
master_t = dict(
    KSM = 1e12,
    BTC = 1e8,
    KBTC = 1e8,
    KINT = 1e12,
)

def fix_currency(row):
    token = row['token']
    divisor = master_t.get(token, 1.0)
    row[token.lower()] = float(row.get('amount', 0.0)) / divisor
    return row



Where did the biggest vaults send their tokens to or receive tokens from?

Let's see who the vault owners got the KINT from to start

In [60]:
xtoken_transfers.rename(columns={"from.id":"from_id", "to.id":"to_id"}, inplace=True)
xtoken_transfers = xtoken_transfers.apply(fix_currency, axis=1)
xtoken_transfers.columns

Index(['amount', 'dot', 'from.karura', 'from.kintsugi', 'from.kusama',
       'from.moonriver', 'fromChain', 'from_id', 'kbtc', 'kint', 'ksm',
       'timestamp', 'to.karura', 'to.kintsugi', 'to.kusama', 'to.moonriver',
       'toChain', 'to_id', 'token'],
      dtype='object')

In [61]:
xtoken_transfers.timestamp.max()

'2022-06-02T18:36:54.441000Z'

There are a few accounts involved in crowdloans and other "shenanigans". Let's label them, too. 

In [62]:
shenanigans = xtoken_transfers.loc[xtoken_transfers.toChain==2092].groupby('from_id').agg(dict(kint=sum, to_id='nunique'))
shenanigans.sort_values('to_id', ascending=False).iloc[0:20]

,kint,to_id
from_id,,
a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,124102.741526,518
a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,7634.166115,440
a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,54388.852580,162
a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,6716.357000,100
a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,46032.498224,72
a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,82.154192,18
a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,277.394270,13
a3aPp8Ei4Qm3dTr57rRabVJhFKpGdzdxYkgFFgqVNZ7nqudyw,3.100000,12
a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,346.266419,6


That analysis seems to suggest we should label the ones which sent more than 5 KINT as "shenanigans"...


In [63]:
shenanigans_list = shenanigans.loc[shenanigans.to_id>5].index
shenanigans.reset_index(inplace=True)
enrich_df(shenanigans, 'from_id')

add_label_to_list(shenanigans_list, "shenanigans")
enrich_df(shenanigans, 'from_id')
shenanigans.sort_values('to_id', ascending=False).iloc[0:20]


,from_id,kint,to_id,label_from_id
39,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,124102.741526,518,shenanigans
529,a3drECEeNJTEfPkMYCu1njch4tmMK1msjjMPbFsF7PSe5Qn5D,7634.166115,440,shenanigans
229,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,54388.852580,162,shenanigans
381,a3cvxyxiBBT88FkVZ7BYyhKPL9sNVdXYk6UBqk8GQnYzrK6xW,6716.357000,100,shenanigans
320,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,46032.498224,72,shenanigans
534,a3dtigkC2uf411XjMBN4KE9WSaNQUV3TxhkoRgYtj2VRHYHkT,82.154192,18,shenanigans
248,a3btcmyVE6ENtWVyHiX9QnorJfKfA2TsSCF43urDeNAWKueH6,277.394270,13,@seergeist/Top Issuer/Selfish/Self Issuer/shen...
24,a3aPp8Ei4Qm3dTr57rRabVJhFKpGdzdxYkgFFgqVNZ7nqudyw,3.100000,12,shenanigans
281,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,346.266419,6,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...
10,a3aGJRwEXw9gUV5DNKhgjcv8iiThf8WZ1TtGXxBJRVU2bZLzq,203.652000,6,shenanigans


Who are the accounts that have sent KINT or KSM to the top 20 vaults? We shall exclude "shenanigans".

to_id: the top 20 vaults

from_id: the accounts sending KINT excluding shenanigans



In [64]:
from tensorflow.python.profiler import trace
funding_accounts = xtoken_transfers.loc[
                                        (xtoken_transfers.to_id.isin(vaults.iloc[0:20, 0])) & 
                                        ~(xtoken_transfers.from_id.isin(shenanigans_list)) & 
                                        (xtoken_transfers.toChain==2092)
                                        ].groupby(['to_id', 'from_id', 'from.kusama','fromChain']).agg(dict(kint=sum, ksm=sum, timestamp=min))
funding_accounts.reset_index(inplace=True)
funding_accounts.rename(columns=dict(to_id="vault", from_id="daddy"), inplace=True)
funding_accounts.sort_values('timestamp')


,vault,daddy,from.kusama,fromChain,kint,ksm,timestamp
6,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,Ffj4XHRejwspbtpcWZHCrTpGVLSzsvynRPdS4N5Z8eajsrK,2092,11.00000,0.000000,2022-03-23T17:19:54.491000Z
13,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,2092,15.00000,0.000000,2022-03-23T17:20:12.525000Z
18,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,a3adPv7McJ11sBVApm9ZcR52wpQAKBLFjd2n9E6DH1cBkccMg,CzZTxaAvBSMW7PKHhfS7qFd3Bgo5Sw8n8tT4kQcGx2hRCe1,2092,10.00000,0.000000,2022-03-27T16:55:12.407000Z
17,a3eZhSC12zE4D49ir4QkxZwDC3jU6iwNQ953ZZzQsCLTnPJjs,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,JKbZuQCY1Z7H5HJ1fXDobKbt29v8sFyyLXQuqdQ7axQtzdH,2092,210.00000,110.660000,2022-03-28T10:07:36.474000Z
1,a3addPTx9ngWGKq3dguw7vs7NA2PimcDUHWJ32HsuoFL74zdo,a3fxS24Bduq8cxT4oV7RQ6q6vfEdSEkaapER75BS4rF7U6Q63,JKbZuQCY1Z7H5HJ1fXDobKbt29v8sFyyLXQuqdQ7axQtzdH,2092,14.90000,91.948160,2022-03-28T12:46:12.326000Z
19,a3fxCiEXfZJThTpHR5FPT3jngnFWQygAtw1ieDp2w9669v6eN,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,2092,10.00000,0.000000,2022-03-29T16:31:48.447000Z
10,a3dCFNgcQsgL4wX6apeaxVLTyRzeCiUdEczyvPcXZ5Dmg5rez,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,2092,10.00000,0.000000,2022-03-29T16:33:12.651000Z
5,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,E7auMi9fNToJ9hkTDWttd3jFriaYx3FD8ezzmGUaKAQnpzh,2092,1.00000,0.000000,2022-04-01T02:06:18.439000Z
11,a3dqzCRPDD9gwrXZ63FBa6hX2q6DsMcUNWPooK2mNNwTDqajq,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,E7auMi9fNToJ9hkTDWttd3jFriaYx3FD8ezzmGUaKAQnpzh,2092,2.00000,0.000000,2022-04-09T03:59:54.847000Z
14,a3eKvTxY56smUwHU9vLpw9w5kSqpoPkJskU2tNxUSHAnntQTS,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,EcdZSvpYwbhESMviUwYQb9WiZTgnTYrAsRLUf3GmXyzy5dH,2092,6.50000,0.940000,2022-04-23T18:58:00.398000Z


Let's label each daddy with his vault

In [65]:
for n, row in funding_accounts.iterrows():
  vault = row['vault'][-5:]
  daddy = row['daddy']
  add_label_to_list([daddy], f'Daddy of {vault}')  
  add_label_to_list([row['vault']], f'{vault}')

In [66]:
add_label_to_list(funding_accounts.daddy, "Likely Vault Owner")
enrich_df(funding_accounts, 'vault')
enrich_df(funding_accounts, 'daddy')
funding_accounts.sort_values('ksm', ascending=False).loc[:, ['label_vault', 'label_daddy', 'kint', 'ksm']]


,label_vault,label_daddy,kint,ksm
9,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.00000,142.360000
17,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Simon Kraus/@sik/Top Redeemer/Top Issuer/Selfi...,210.00000,110.660000
1,Simon Kraus/@sik/Top Issuer/Selfish/Top 20 Vau...,Simon Kraus/@sik/Top Redeemer/Top Issuer/Selfi...,14.90000,91.948160
7,Social/Top 20 Vault/Daddy of Poxg8/rEszH/Likel...,Social/Top 20 Vault/Poxg8/Daddy of rEszH/Likel...,6.80341,50.335594
8,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...,Daddy of eCQvW/Likely Vault Owner,0.00000,30.000000
15,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Simon Kraus/@sik/Top Issuer/Selfish/Top 20 Vau...,438.00000,24.000000
0,Simon Kraus/@sik/Top Issuer/Selfish/Top 20 Vau...,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...,0.00000,3.110000
16,Simon Kraus/@sik/Top 20 Vault/Self Issuer/nPJjs,Top Redeemer/Top Issuer/Daddy of 74zdo/Daddy o...,0.00000,3.110000
14,@mafux777/Top Issuer/Selfish/Top 20 Vault/Self...,Top Redeemer/Daddy of ntQTS/Likely Vault Owner,6.50000,0.940000
12,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ9g,Daddy of ceZ9g/Likely Vault Owner,0.35000,0.000000


Of the funders, which ones do we know from the KSM analysis?

In [87]:
k = kusama_transfers.loc[kusama_transfers.ksm>50]
add_label_to_list(k.kintsugi, 'K>50')
enrich_df(k, 'kintsugi')
k

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,ksm,first_seen,kintsugi,label_kintsugi
0,GDN8YBtPVRkjR7YtEKMUMXzBdtsNYWW7djdsxcQDqSZSxmM,10000.00000,2022-04-25T15:53:42.020Z,a3drCagyKmK1GQnu4MgDXmbKJxrNPURq6xXdL3JR4xVbceZ9g,Top Issuer/Social/Top 20 Vault/Self Issuer/ceZ...
1,HExSfZUFJPkbUxpKQ5jM54TA7ThwfsCRBfgowmzzCcH8rV2,3446.75000,2022-05-02T15:20:06.029Z,a3esntpLud7yGGrkKnqyueJqmwKwE3ZBoG5ZNyHafirmLLDD3,Top Redeemer/Top Issuer/Selfish/Top 20 Vault/S...
2,CrZxfsquEZeXLZUk7Uxdts8NpKz1oc7pdDXwHYCSeJ8SLgW,2940.17498,2022-04-26T20:29:06.018Z,a3aVQQpfHH49ACiLzDZP8w8eTA2oW7gvifX7E6dLsBJTBdzVR,Self Issuer/K>50
3,GcohCErDM7LjUfBkmNqu19CNzRUEp84DQP4nZeUhjrBHqNP,847.66346,2022-04-25T17:29:30.016Z,a3eFe9M2HbAgrQrShEDH2CEvXACtzLhSf4JGkwuT9SQ1EV4ti,@paride/Social/Top 20 Vault/K>50
4,Ea9GXVeSWj1vxXutpMPtZAinuACMeWYyVfKHqKrn2gb11kj,826.38600,2022-04-26T07:15:42.025Z,a3cCyigH5pLJXcLKRNGFaBnx3a7diTXq9pPZ1TB8XWgqeCQvW,@spazcoin/chaos DAO/Top Redeemer/Top Issuer/To...
5,Cdcb6dodh71D8rCf3sWsokuXGkaS1YKcA7mgGWbDuSrHSFA,714.00000,2022-05-16T14:33:36.024Z,a3aGT3FRF1WgWtWdi8VmhB3YEJVE6XtrvT41TqcKFxZbuUvS1,@warinelly/Social/Top 20 Vault/K>50
6,EcdZSvpYwbhESMviUwYQb9WiZTgnTYrAsRLUf3GmXyzy5dH,581.95000,2022-04-25T10:08:12.007Z,a3cFU1biFvmBCup9SBvqihpvqVmwCtLsT1mK2dzqwWC94AjJw,Top Redeemer/Daddy of ntQTS/Likely Vault Owner...
7,DMZC2aCdR5SjYTr9duiHFHWHSuqwfFUjHX1Qru9YLSe9H5e,579.79000,2022-05-01T20:50:54.012Z,a3azPeBMe1EexQvFMd5otaV4q4fPN3Ya5aBQhaChpGzbhLPpe,pumpernickel/Social/Top 20 Vault/Self Issuer/h...
8,GCE5wFsJX3x3MpPrekqnXe2STTye3gdvxvjX5gSXL5YSZ74,484.50000,2022-05-20T19:26:54.016Z,a3dq4Y63JgLdTijbuL6f25mRMDfwVjw1EmrpRgRV7FzEbdxDB,Social/Top 20 Vault/K>50
9,DPe86fQfixDTfejAiEJbRt2mvbtcGkWdGXsyCFDu37iKYVu,453.50000,2022-05-18T11:01:30.020Z,a3b2UaFSr3YXj93SEeA8UtffMZ95QiA57UARa8Y3tdhGmWr9h,K>50


In [88]:
add_label_to_list(top_loaners.kintsugi, 'Top Crowdloaner')
enrich_df(top_loaners, 'kintsugi')
top_loaners.sort_values('ksm', ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,kusama,account.substrateAccount,ksm,first_seen,kintsugi,label_kintsugi
643,EwAhYwv7TJ1V8fxchsyZHNyturGYgoxhfEiiEnwCt7bWFjD,5ERZ3Ec3V6H5jUrWr14vfL1ykKa2k1znFHPyKaWyj5u6mCfs,22688.00000,2021-09-24T10:52:42.022Z,a3ca19hjMVGsXAWTU69n9rXAJg8Knea8ZYZ8QsabbwYGehfYx,Top Crowdloaner
1439,HpofvM4WwTEfiAfD8tJPSoV4EQtUxHCbUK8eSHcweoudWS1,5HKC1c1BtaSJv4MDSS5FVVSUue8egHU296UPFn1fTrbQtTYd,18517.00000,2021-09-24T10:53:18.003Z,a3fTe858Vtm2kM5xAganUggaoqStQaqboSNCpon6HgJxxpnhW,Top Crowdloaner
596,EkmdfH2Fc6XgPgDwMjye3Nsdj27CCSi9np8Kc7zYoCL2S3G,5EF9yLw9dF5bvjrnAevvk61sV8jsPXdXhQyNvwr34zyqHC4W,17270.44313,2021-09-21T23:42:36.005Z,a3cPc5p4TdRg3MmTjQoe9wHACQwVdJ5mJzghpUwvfHTMPDXvg,Top Crowdloaner
520,EVAMXLuZGhXcjPWb3JgX6a5qfPq2FH9usUZNuzW5XNQGvEw,5DyYhD12vugbs5a4pLVdd9D5h57bDaTyTVdozFiYbj9uXow7,15001.00000,2021-09-17T06:32:06.005Z,a3c7zog8Lw6H3J7B24VCrpLMQcssM88bkkmNFYFoApBXTU9FC,Daddy of ceZ9g/Likely Vault Owner/Top Crowdloaner
1295,HH5CgPv1RAgQMgpJh6p3NdT9juW5RkTgNTZyY14x3YEEY9W,5GmTYN43P49kehsNXzHm9RGT19dGGkwHDzcpasj7UFKjVW2J,10000.00000,2021-09-16T11:15:06.007Z,a3euueqBMPEkC5jUKn8zzLcQmvxP2BK54XGMG8sojghhHS8R1,Top Crowdloaner
...,...,...,...,...,...,...
214,DKHXv2J3o9DWZwuNC8dKkRn4ycpu6ikiUTRnYSc1rAXJxmT,5CofsbgRRS8Hkv8TbVKaRo4mvPLb6RuaG6cgPtAeY3x2ZuW9,3.05000,2021-09-25T18:32:24.006Z,a3ax7z4ojRcijBwjQqe2oczD6rC6Lzz3MZNM7wtFGkWKaWF7N,Top Crowdloaner
851,FfjPRZZNzgvQEJWJes4ojqNpUnP4CQhmvJAh8QCfLutsrL4,5FA7j7DgkdfzeaV4Xx41unUNftW9FXbXKYTRJU8FBYhQ8hPf,3.05000,2021-09-25T10:54:48.008Z,a3dJZqaLzkpGS5c61n6mF6ychbhFuA5jJcpBrrUCsQ14x58XX,Top Crowdloaner
1124,GhF5ETmUupvonU6Uzt6HgJntYMsEwbCiiVC14pQqTL4xBGU,5GBdQv7trYp148eeiJ53Piwnjx5dSGn2GLeScQYTMf7aCyok,3.04570,2021-09-24T19:57:06.012Z,a3eL5XPFCrjQSVAFbxSnGav67fkqPLpuoZcNtAQd5a7V89VgS,Top Crowdloaner
105,CuvFiWWHSSdnHEqnyejt1aMdefca1cNBe92rbB5GJ9TtKPu,5CQJbQAdf5Ri2dRQ2Gqgz4DMV4PNmLoBjGJHTvu7nVvy96sB,3.04000,2021-09-24T22:29:36.014Z,a3aYkhsHwfG29Tf2MGRYvBFMgQs98ftvy2Y2j1vyjzxJX5ZWd,Top Crowdloaner


What do people do with their KBTC?

In [36]:
my_currencies = ['kint', 'ksm', 'kbtc']

# def add_more_labels(my_currency):
agg = {}
for c in my_currencies:
  agg[c] = sum

top_transfer = xtoken_transfers.groupby(['from_id', 'fromChain','to_id', 'toChain']).agg(agg)
top_transfer.reset_index(inplace=True)


for my_currency in my_currencies:
  q = top_transfer[my_currency].quantile(.8)

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].from_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Mover')

  top_transfer_ids = top_transfer[top_transfer[my_currency]>q].to_id
  add_label_to_list(top_transfer_ids, f'Top {my_currency.upper()} Sink')

enrich_df(top_transfer, 'from_id')
enrich_df(top_transfer, 'to_id')

top_transfer.sort_values('kbtc', ascending=False)


,from_id,fromChain,to_id,toChain,kint,ksm,kbtc,label_from_id,label_to_id
1477,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2000,0.00000,0.0,3.273347,@alibaba/Top Redeemer/Top Issuer/Top KINT Move...,@alibaba/Top Redeemer/Top Issuer/Top KINT Move...
1461,a3dJZWg4s2ZXPVygL5xTTW6F93hoy6mFadKHKbQAkHnodw9uR,2092,a3bkRMWVb3C3JyXVFvfR5Brq435C6eqMr42YhA759JyKTzRzQ,2092,11.00000,0.0,1.735892,Top Issuer/Daddy of TzRzQ/Likely Vault Owner/T...,Top Redeemer/Social/Top 20 Vault/Self Issuer/D...
1473,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0x63e33447838b9025e5cfdb35d4acc12a38342614,2023,0.00000,0.0,1.649605,@alibaba/Top Redeemer/Top Issuer/Top KINT Move...,Top KBTC Sink
637,a3ar4dXg4JKaQ939WQCcv38xUFwU3PuW1FPuzsGAgC2GFYPxK,2092,a3dMJSmFcqTDpvRPfM2HKn7CHd5uw3G7atogtxXeXru3LGURE,2092,0.50000,0.0,1.348521,Top Issuer/Top KBTC Mover/Top KBTC Sink,@alibaba/Top Redeemer/Top Issuer/Top KINT Move...
1509,a3dV3XyhVP3f1B1dwCvW7V8k87M2maqRxLpk8bb5f4TbPVSFD,2092,0x7da0419a1c47750f19440019cda780998b32268f,2023,2798.68100,0.0,1.053451,Top Issuer/Top KINT Mover/Top KINT Sink/Top KB...,Top KINT Sink/Top KBTC Sink
...,...,...,...,...,...,...,...,...,...
825,a3biTZzhnZEcTHRUnWNnFV5VDAXsGDbj1SzW4D5Er89ysLQ8J,2092,0xe7fd369ef0ffe51d8de0708f412e42fae6bcd9cc,2023,49.00000,0.0,0.000000,,
826,a3biaUj1LLojHEKPoKmRdZnikTUv4YGNwQ2g7MFNu3Z2nUoZC,2092,a3fVzyUt9RGMLbf3dD5C11GybmJV8mHxKuyKqodhdBbmZuUrY,2092,48.89500,0.0,0.000000,,Top KINT Mover/Top KINT Sink/Top KBTC Mover/To...
827,a3bifWLgkU3vPugAfCjG5YwF68ghb5TFpCK587qUgdKnA81sw,2092,a3dLDEmtTHD1k8uc7rtAcRntz7frvkkPTarcgYBPo43FArqQW,2092,2.20757,0.0,0.000000,,
828,a3bigmuVMKUCrDgirYfs8mda7UNMaUPpJJ65tf6rfF2iw9VRU,2092,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,0.99987,0.0,0.000000,,shenanigans/Top KINT Mover/Top KINT Sink


Same story, but with KINT - removing two accounts which seem to be system accounts

In [39]:
agg['to_id']='nunique'
agg['timestamp']='min'
xtoken_transfers['last_seen'] = xtoken_transfers.timestamp # workaround to agg this col, too
agg['last_seen']='max'

top_transfer_from = xtoken_transfers.groupby(['from_id', 'fromChain','toChain']).agg(agg)
top_transfer_from.kint = top_transfer_from.kint.apply(lambda k: round(k))
top_transfer_from = top_transfer_from.reset_index(inplace=False)

exclude = top_transfer_from.sort_values('kint', ascending=False).iloc[0:2, 0]
list(exclude)
add_label_to_list(exclude, 'KINT System?')
enrich_df(top_transfer_from, 'from_id')

In [40]:
top_transfer_from.sort_values('kint', ascending=False)

,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id
407,a3cgeH7D28bBsHWJtUcHf7srz25o34gCKi8SZVjky6nMyEm83,2092,2092,1325000,0.0,0.000000,2,2022-03-16T20:11:24.342000Z,2022-03-24T20:29:54.518000Z,Top KINT Mover/KINT System?
45,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,2092,2092,114536,0.0,0.000000,514,2022-02-21T13:48:30.573000Z,2022-05-31T15:59:30.484000Z,shenanigans/Top KINT Mover/Top KINT Sink/KINT ...
266,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,54347,0.0,0.000000,162,2022-02-20T00:37:12.309000Z,2022-05-30T17:19:00.407000Z,shenanigans/Top KINT Mover/Top KINT Sink
371,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,46032,0.0,0.000000,72,2022-02-17T17:37:06.708000Z,2022-05-26T01:42:06.333000Z,shenanigans/Top KINT Mover
254,a3bi5krmz6NvNbpipDiVmpUtzPp3qa5RLgFDUYnhA6RLvXoyz,2092,2092,35009,0.0,0.000000,1,2022-05-04T10:10:00.284000Z,2022-05-04T13:41:06.603000Z,Top KINT Mover/Top KINT Sink
...,...,...,...,...,...,...,...,...,...,...
509,a3dLJ8xMM8SsbC38pKykrqKPS5upCzni3QugSn3eMbUtUESX9,2092,2023,0,0.0,0.124932,1,2022-05-17T05:38:00.543000Z,2022-05-17T05:38:00.543000Z,Top Issuer/Top KBTC Mover/Top KBTC Sink
508,a3dLJ8xMM8SsbC38pKykrqKPS5upCzni3QugSn3eMbUtUESX9,2092,2000,0,0.0,0.397863,1,2022-05-11T18:20:42.472000Z,2022-05-17T17:11:06.480000Z,Top Issuer/Top KBTC Mover/Top KBTC Sink
507,a3dJmVwmyQy9Wscq9Vokz9yw7coy6epe3nW94R6x1xwCrCXSH,2092,2000,0,0.0,0.008065,1,2022-05-08T01:23:06.486000Z,2022-05-08T01:23:06.486000Z,Top KBTC Mover/Top KBTC Sink
505,a3dJcbrvGrFBfPdN6DBjwgRGbMDdQCr2YFAgNkPzRJA2aRSPi,2092,2092,0,0.0,0.000000,1,2022-03-26T10:39:54.452000Z,2022-03-26T10:39:54.452000Z,


In [ ]:
def calc_secs(row):
  return (pd.Timestamp(row['last_seen'])-pd.Timestamp(row['timestamp'])) // pd.Timedelta("1s")

In [ ]:
enrich_df(top_transfer_from, 'from_id')
top_transfer_from['duration'] = top_transfer_from.apply(calc_secs, axis=1)
# top_transfer_from.loc[top_transfer_from.kbtc>0.025].sort_values('duration', ascending=False)
top_transfer_from.sort_values('duration', ascending=False)
# top_transfer_from.sort_values('timestamp', ascending=True)

,from_id,fromChain,toChain,kint,ksm,kbtc,to_id,timestamp,last_seen,label_from_id,duration
357,a3cUcEFSfpaK8spLYmzshHq1XcmyMtrJf66R1r84ngdQFiaXC,2092,2092,46032,0.0,0.0,72,2022-02-17T17:37:06.708000Z,2022-05-26T01:42:06.333000Z,shenanigans/Daddy of kccMg/Likely Vault Owner/...,8409899
724,a3ec2HDHg824rqKYqXW3v5Np3z9XKhZkZZb9YdxRYJd9QVLqe,2092,2092,534,0.0,0.0,1,2022-02-17T18:24:00.183000Z,2022-05-24T16:30:37.096000Z,Top KINT Mover,8287596
256,a3bkd4Z7FSnAVABG9JJV8UyESACpKgretzxMbYx7zLTLoVj2p,2092,2092,53996,0.0,0.0,159,2022-02-20T00:37:12.309000Z,2022-05-26T16:33:18.239000Z,shenanigans/Daddy of hLPpe/Daddy of EV4ti/Like...,8265365
45,a3aWepSsLfy67vUAAaKCiSnCj7G72feeTCgfKCBLivBWWTdFW,2092,2092,105560,0.0,0.0,509,2022-02-21T13:48:30.573000Z,2022-05-26T12:40:12.332000Z,shenanigans/Daddy of uUvS1/Daddy of Poxg8/Like...,8117501
897,a3fqG9iWWv15rxGRviVutvjLEy9Dk42Gm9mNoNzPVbUKtfKWP,2092,2092,658,0.0,0.0,2,2022-02-20T21:13:30.212000Z,2022-05-24T14:33:54.277000Z,Top KINT Mover,8011224
...,...,...,...,...,...,...,...,...,...,...,...
376,a3ccJQnR8VfG2YJ3981B5iSvyZz1AXw7bCxwFu6vmbPmXXnSM,2092,2092,1,0.0,0.0,1,2022-03-29T19:56:30.983000Z,2022-03-29T19:56:30.983000Z,,0
378,a3cctiyjzo6QAB6CHvBL5CUs6Aq4U8H8bSuiwsj499Pa7J7sZ,2092,2092,2,0.0,0.0,1,2022-03-07T11:11:24.370000Z,2022-03-07T11:11:24.370000Z,,0
381,a3cdNKCsRmr1KTCHH2W3V5HWqHwP88RmShhd1zo9eBMrFqbtq,2092,2092,5,0.0,0.0,1,2022-04-08T04:49:30.173000Z,2022-04-08T04:49:30.173000Z,,0
383,a3cdPu3pFVPkwH1PYicAWKPKK4spLsecdGAXmDNzJjP7Z9fxm,2092,2092,9,0.0,0.0,1,2022-04-07T07:37:12.212000Z,2022-04-07T07:37:12.212000Z,,0
